# Script Writer Workflow Using CrewAI


In this notebook, we demonstrate how to build and collaborating agentic AI system using **CrewAI**. The system integrates multiple agents, each with a specific role, to:
1. Gather current information from the internet about given topic.
2. Make a conversational script out of the information.

---
### Workflow Overview
1. Define tools to support the agents, such as web search capabilities.
2. Create two agents for data gathering and script creation.
3. Define tasks corresponding to these agents.
4. Set up a crew that executes the tasks sequentially.
5. Run the crew to achieve the desired outcome.

---
### Code Walkthrough
Below is the implementation.


### Step 1: Import Required Libraries and Define Tools

In [1]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai==0.95.0
!pip install crewai-tools==0.25.8
!pip install duckduckgo-search
!pip install langchain-community
!pip install litellm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 10.0 MB/

In [2]:
from crewai import Agent, Task
from crewai.tools import tool
from crewai.llm import LLM
from langchain_community.tools import DuckDuckGoSearchRun
from datetime import datetime
import os

# Current date for context
Now = datetime.now()
Today = Now.strftime("%d-%b-%Y")

# Define a web search tool
@tool("DuckDuckGo Search")
def search_tool(search_query: str):
    """Search the internet for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)


### Step 2: Define the Agents

In [3]:
# Alternative Search tool from Crew AI tools. Try this and see the difference
from google.colab import userdata
from crewai_tools import EXASearchTool
import os

os.environ['EXA_API_KEY'] = userdata.get('exa_api_key')

Exa_Search_Tool = EXASearchTool ()

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'example'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https:

In [4]:
from crewai import Agent
#from langchain_community.tools.exa.tool import ExaSearchTool

## Agent designed to gather information about a topic from internet.
content_explorer = Agent (
    role = 'content explorer',
    goal = 'Gather and provide latest information about the topic from internet',
    # llm = 'gemini/gemini-2.0-flash-lite',
    # llm = 'groq/gemma2-9b-it',
    llm = 'openrouter/google/gemini-2.5-flash',
    # llm = 'gpt-4o',
    verbose = True,
    backstory = ('You are an expert researcher, who can gather detailed information about a topic.\
                  Gather at least 10 information. Consider you are on : '+Today),
    tools = [Exa_Search_Tool],
    cache = True,
    max_iter = 5
)

## Agent to create a script out of information.
## Back story sets the context for the agent.
# script_llm = LLM (model='groq/llama3-70b-8192', temperature=1.2)
script_llm = LLM (model='gpt-4o', temperature=0.9)
Script_Writer = Agent (
    role = 'Script Writer',
    goal = 'With the details given to you create an interesting conversational script out of it',
    #llm = 'openrouter/deepseek/deepseek-chat-v3-0324:free',
    llm=script_llm,
    verbose = True,
    backstory = ('You are an expert in Shakesperean literature. You are very good in creating conversations with the given chain of information.\
                 Tell as a script in 200 words.\
                 Consider you are on '+Today),
)

os.environ['OTEL_SDK_DISABLED'] = "True"


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Step 3: Define the Tasks

In [5]:
from crewai import Task

# Task to gather Latest information
get_details = Task (
                      description = "Get latest, trending, interesting information and news about {topic}",
                      expected_output = "Latest news, interesting information and trivia about {topic}",
                      agent = content_explorer
                    )

## Task to create script.
create_a_script = Task (
                      description = "Considering the given details in time order make an interesting conversation",
                      expected_output = "A humorous conversation connecting key details",
                      agent = Script_Writer,
                      context = [get_details],
                      output_file = 'Script.txt'
                    )



### Step 4: Set Up the Crew

In [6]:
from crewai import Crew, Process
from datetime import datetime

# Callback function to print a timestamp
def timestamp(Input):
    print(datetime.now())

# Define the crew with agents and tasks in sequential process
crew = Crew (
    agents = [content_explorer, Script_Writer],
    tasks = [get_details, create_a_script],
    verbose = True,
    Process = Process.sequential,
    step_callback = timestamp
)

### Step 5: Run the Crew and Observe Results

In [7]:
# Set your OpenAI API key or any other LLM API key
from google.colab import userdata

# Set your Groq API key or any other LLM API key
# os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('openai_key')
os.environ['OPENROUTER_API_KEY'] = userdata.get('openrouter_api_key')
# os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

# Run the crew with a specific stock
result = crew.kickoff (inputs={'topic' : "US India Tariffs"})


# Agent: content explorer
## Task: Get latest, trending, interesting information and news about US India Tariffs
2025-09-20 05:58:48.169040


# Agent: content explorer
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US India tariffs latest news September 2025\"}"
## Tool Output: 

Search results: 

2025-09-20 05:58:48.169313
2025-09-20 05:58:58.813308


# Agent: content explorer
## Final Answer: 
**Latest News, Interesting Information and Trivia about US-India Tariffs (as of September 20, 2025):**

1.  **US-India Trade Dialogue: Focus on Critical Minerals and EV Tariffs (Sept 18, 2025):** Discussions are currently active between the US and India focusing on tariffs related to critical minerals crucial for electric vehicle (EV) production. The US is pushing for enhanced access to India's critical mineral processing capabilities, with potential reciprocal tariffs on finished EV goods also on the table.
2.  **India's New Tariff Schedule: Impact on US Agricultural 

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



---
### Conclusion
This workflow demonstrates how to integrate multiple agents to perform tasks sequentially in a context. It showcases the power of CrewAI for building AI-driven collaborative solutions.
